# TestQuery Notebook

This is a test notebook to accompany the following post:

[
![medium-post](../../priv/images/medium-post-jupyter-notebooks.png)
](https://https://medium.com/@tonyhammond/latest)

In this notebook we shall be using [IElixir](https://github.com/pprzetacznik/IElixir) which implements a Jupyter kernel for Elixir. We'll also be using the Elixir package [SPARQL.Client](https://hex.pm/packages/sparql_client) which provides support for SPARQL querying over RDF datastores. For further information on notebook format and architecture see the [Jupyter Notebook](http://jupyter.org/) project site.

[
![jupyter](../../priv/images/jupyter.png)
](http://jupyter.org/)

Here's a quick overview of our action plan:

1. Creating the project
2. Setting up the environment
3. Running basic SPARQL.Client queries
4. Installing our TestQuery modules
5. Testing it out


## _A couple caveats first_

* _I am still very new to all this and not sure how best to configure IElixir for `Mix` projects. As a consequence I wasn't able to get the project tree and attributes loaded automatically but had to resort to loading the modules individually. I also wasn't able to use the `Mix` `config.ex` file which would have allowed me to select for an alternative HTTP client, e.g. `hackney`, instead of the default `httpc` client. This meant that the default HTTP method and protocol needed to be overridden._

* _That aside, it all seems to work well enough. Although I am pretty sure it could be made to work even better._ 

## 1. Creating the project

We're going to be working with the `test_query` project from an earlier post:

[
![medium-post](../../priv/images/medium-post-querying-rdf.png)
](https://medium.com/@tonyhammond/querying-rdf-with-elixir-2378b39d65cc)

Now, we may have to make some changes to the original `TestQuery` modules so we'll make a copy of the `test_query` project and name this new project as `test_ipynb`.

We'll also name this notebook as `test_query.ipynb` and keep it under a new `priv/notebooks/` directory.

## 2. Setting up the environment


&#x279C; ** Create `sparql_env`.**

IElixir uses the concept of virtual environments for managing packages. It uses [`Boyle`](https://github.com/pprzetacznik/IElixir#package-management-with-boyle) as its package manager.

Let's first create a `sparql_env` environment for our SPARQL dependencies using `Boyle.mk/1`. (Note also that there is a previously created an `rdf_env` environment set up for separately exploring [RDF.ex](https://hex.pm/packages/rdf) which we can just ignore.)

In [1]:
Boyle.mk("sparql_env")

All dependencies up to date


{:ok, ["rdf_env", "sparql_env"]}

&#x279C; **Activate the environment.**

Next step is to activate the environment which will take care of compiling.

In [2]:
Boyle.list

{:ok, ["rdf_env", "sparql_env"]}

In [3]:
Boyle.activate("sparql_env")

All dependencies up to date


:ok

And we can double check that this is the current environment.

In [4]:
Boyle.active_env_name

"sparql_env"

&#x279C; **Install `sparql_client` dependencies.**

Next we install out dependencies.

In [5]:
Boyle.install({:sparql_client, "~> 0.2.1"})

Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  content_type 0.1.0
  decimal 1.5.0
  elixir_uuid 1.2.0
  jason 1.1.2
  json_ld 0.3.0
  mime 1.3.0
  nimble_csv 0.4.0
  rdf 0.5.1
  sparql 0.2.5
  sparql_client 0.2.1
  sweet_xml 0.6.5
  tesla 1.1.0
All dependencies up to date


:ok

&#x279C; **And check our package.**

And lastly let's just sanity check that we do indeed have the `SPARQL.Client` module installed.

In [6]:
exports SPARQL.Client

__adapter__/0               __middleware__/0            default_accept_header/1     
delete/1                    delete/2                    delete/3                    
delete!/1                   delete!/2                   delete!/3                   
get/1                       get/2                       get/3                       
get!/1                      get!/2                      get!/3                      
head/1                      head/2                      head/3                      
head!/1                     head!/2                     head!/3                     
options/1                   options/2                   options/3                   
options!/1                  options!/2                  options!/3                  
patch/2                     patch/3                     patch/4                     
patch!/2                    patch!/3                    patch!/4                    
post/2                      post/3                      post/4   

Looks good.

## 3. Running basic SPARQL.Client queries

&#x279C; **Let's try out a simple query.**


Let's choose a SPARQL endpoint. The ever-trusty [DBpedia](https://wiki.dbpedia.org/) will make for a good fallback.

In [7]:
service = "http://dbpedia.org/sparql"

"http://dbpedia.org/sparql"

And we'll create a basic SPARQL query to retrieve one triple.

In [8]:
query = "select * where {?s ?p ?o} limit 1"

"select * where {?s ?p ?o} limit 1"

&#x279C; **First query try gives error.**

In [9]:
SPARQL.Client.query(query, service)

{:error, "unsupported result format for select query: \"text/html; charset=UTF-8\""}

&#x279C; **Let's check out the documentation.**

Uncomment this line and run to see the help documentation for `SPARQL.Client.query`.

In [10]:
# h SPARQL.Client.query

nil

We see that there are three supported access methods:

* `GET`, `SPARQL 1.1`
* `POST` (URL-encoded), `SPARQL 1.0` -  _default_
* `POST`, `SPARQL 1.1`

Since the default URL-encoded `POST` method does not work for this `service`, let's try the `GET` method.

&#x279C; **Second try succeeds – after setting :request_method (and :protocol_version).**

In [11]:
SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

{:ok, %SPARQL.Query.Result{results: [%{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid>}], variables: ["s", "p", "o"]}}

Hey, that works! Let's run it again and capture the result this time.

In [12]:
{:ok, result} = SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

{:ok, %SPARQL.Query.Result{results: [%{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid>}], variables: ["s", "p", "o"]}}

In [13]:
result |> SPARQL.Query.Result.get(:p) |> IO.inspect

[~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>]


[~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>]

Time for something more interesting. Let's try out our earlier `TestQuery` package.

## 4. Installing our TestQuery modules

&#x279C; **Install TestQuery**

Is our `TestQuery` module loaded?

In [14]:
exports TestQuery

UndefinedFunctionError: 1

That'll be a no, then. So, let's explicitly import the module. We'll try calling the original `TestQuery` module first from the `test_query` project.

In [14]:
import_file("../../tonyhammond/examples/test_query/lib/test_query.ex")

Protocol.UndefinedError: 1

Rats!

Turns out that the call `:code.priv_dir(:test_query)}` is failing. This is aiming to locate the `priv/` directory for the application `:test_query`, but as we did not manage to load this through `Mix` we haven't created this application.

Let's just replace that with a relative path in our `test_ipynb` project copy.

```
  # @data_dir "#{:code.priv_dir(:test_query)}/data/"
  @data_dir "../tonyhammond/examples/test_ipynb/priv/data/"
```

And let's try again but this time reading from the `test_ipynb` project.

In [14]:
import_file("../../tonyhammond/examples/test_ipynb/lib/test_query.ex")

{:module, TestQuery, <<70, 79, 82, 49, 0, 0, 7, 68, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 191, 0, 0, 0, 18, 16, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 8, 95, 95, 105, 110, 102, ...>>, {:query, 2}}

And let's sanity check.

In [15]:
exports TestQuery

data/0      query/0     query/1     query/2     


Success!

&#x279C; **Install TestQuery.Client.** 

Now let's import the `TestQuery.Client` module, again reading from the original `test_query` project.

In [16]:
import_file("../../tonyhammond/examples/test_query/lib/test_query/client.ex")

Protocol.UndefinedError: 1

Same problem on `query-dir`. Similar fix to the `test_ipynb` project copy this time using an absolute path.

```
  # @query_dir "#{:code.priv_dir(:test_query)}/queries/"
  @query_dir System.cwd <> "/../../tonyhammond/examples/test_ipynb/priv/queries/"
```

And let's try again but this time reading from the `test_ipynb` project.

In [16]:
import_file("../../tonyhammond/examples/test_ipynb/lib/test_query/client.ex")

{:module, TestQuery.Client, <<70, 79, 82, 49, 0, 0, 26, 84, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 223, 0, 0, 0, 71, 23, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 46, 67, 108, 105, 101, 110, ...>>, {:_read_tuple, 1}}

And again sanity check.

In [17]:
exports TestQuery.Client

get_query/0          get_query_opts/0     get_service/0        hello/0              
read_table/1         rquery/0             rquery/1             rquery/2             
rquery_all/0         


&#x279C; **Update TestQuery.Client query functions.** 

But before we leave this we're going to update the `TestQuery.Client` query functions to use the working `:request_method` and `:protocol_version`.

We'll save these into a module attribute `@query_opts` as:

```
  @query_opts request_method: :get, protocol_version: "1.1"
```

We'll then add this as an optional argument to all the `TestQuery.Client` query functions, e.g.

```
  def rquery() do
    SPARQL.Client.query(@query, @service, @query_opts)
  end
```

Also lastly we'll make this replacement in our `hello/0` function:

```
  # result.results |> Enum.each(&(IO.puts &1["o"]))
```

which we will replace by the new `SPARQL.Query.Result.get/1` function.

```
  result |> SPARQL.Query.Result.get(:o)
```

which is a heck of a lot more readable.

And let's reimport the new `test_ipynb` project module.

In [18]:
import_file("../../tonyhammond/examples/test_ipynb/lib/test_query/client.ex")

{:module, TestQuery.Client, <<70, 79, 82, 49, 0, 0, 26, 84, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 223, 0, 0, 0, 71, 23, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 46, 67, 108, 105, 101, 110, ...>>, {:_read_tuple, 1}}

## 5. Testing it out

&#x279C; **Now let's query**

To simplify naming let's just import all the functions from our modules.

In [19]:
import TestQuery

TestQuery

In [20]:
import TestQuery.Client

TestQuery.Client

And let's try out our functions. First our simple 'Hello World' function:

In [21]:
[arg] = hello

[~L"Hello World"en]

In [22]:
arg.value

"Hello World"

Now for a single query test let's just copy a query string from one of our stored queries: `priv/queries/alberto.rq`.

In [23]:
query = """
select *
where {
  bind (<http://dbpedia.org/resource/Hurricane_Alberto> as ?s)
  ?s ?p ?o
}
"""

"select *\nwhere {\n  bind (<http://dbpedia.org/resource/Hurricane_Alberto> as ?s)\n  ?s ?p ?o\n}\n"

And let's try that `query` string on our `rquery/1` function.

In [24]:
{:ok, result} = rquery(query)
result
|> SPARQL.Query.Result.get(:o)
|> List.first
|> IO.inspect

~L"Hurricane Alberto"en


~L"Hurricane Alberto"en

And lastly let's try the multiple queries for hurricanes and store that data in separate ETS tables.

In [25]:
rquery_all

Reading alberto.rq
Writing Elixir.TestQuery.Client.alberto
Reading beryl.rq
Writing Elixir.TestQuery.Client.beryl
Reading chris.rq
Writing Elixir.TestQuery.Client.chris
Reading debby.rq
Writing Elixir.TestQuery.Client.debby
Reading ernesto.rq
Writing Elixir.TestQuery.Client.ernesto
Reading florence.rq
Writing Elixir.TestQuery.Client.florence
Reading gordon.rq
Writing Elixir.TestQuery.Client.gordon
Reading helene.rq
Writing Elixir.TestQuery.Client.helene


[:ok, :ok, :ok, :ok, :ok, :ok, :ok, :ok]

And we can inspect the ETS (Erlang Term Storage) tables by calling the Observer which will open in a separate window as:

In [26]:
:observer.start

:ok

All checks out. &#x1F44D;